# Lecture 9 - Time Series Analysis and Sun Spots

Last lecture we looked in detail at Fourier Analysis and the FFT.

Today we are going to apply that to a real data set. 

The file _/data/dayssn.dat_ contains the daily sun spot number from December 23 1848 until May 31 2015, this is one of longest data recordings in Astronomical Science. 



## The FFT

## The periodgram or power spectra

## Whelch's Method

## The Spectrogram